In [1]:
%load_ext sql
%sql sqlite:///chinook.db

'Connected: @chinook.db'

In [34]:
%%sql
DROP TABLE IF EXISTS Employee;

DROP TABLE IF EXISTS Students;
DROP TABLE IF EXISTS Faculties;

 * sqlite:///chinook.db
Done.
Done.
Done.


[]

In [35]:
%%sql
PRAGMA foreign_keys = ON;

CREATE TABLE Employee(
        id int primary key,
        name varchar(50) not null,
        phone varchar(50),
        title varchar(50) not null,
        manager_id int,
        foreign key (manager_id) references  Employee(id)
);


CREATE TABLE Faculties(
    id integer primary key, 
    name varchar(50) not null,
    unique (name)
);

CREATE TABLE Students(
    id integer primary key, 
    name varchar(50) not null,
    gpa decimal(15,2), 
    faculty_id int, 
    foreign key (faculty_id) references Faculties(id)
)


 * sqlite:///chinook.db
Done.
Done.
Done.
Done.


[]

In [36]:
%%sql
insert into Faculties values (1, 'IT');
insert into Faculties values (2, 'KIB');

insert into Students values (1, 'Sidorov', 3.0, 1);
insert into Students values (2, 'Petrov', 3.5, 2);
insert into Students values (3, 'Verikov', 4.0, 1);
insert into Students values (4, 'Smith', 4.2, 2);
insert into Students values (5, 'Lee', 4.5, 1);
insert into Students values (6, 'Jiw', 3.2, null);

insert into Employee values  (1, 'Vasyugin', '+111', 'Director', null);
insert into Employee values  (2, 'Jones', '+112', 'Vice President', 1);
insert into Employee values  (3, 'Smith', '+113', 'Vice President', 1);
insert into Employee values (4, 'Velaskes', '+114', 'Developer', 2);
insert into Employee values (5, 'Simon', '+115', 'Analyst', 2);
insert into Employee values (6, 'Johnson', '+116', 'Engineer', 3);
insert into Employee values (7, 'Lee', '+117', 'Engineer', 3);

 * sqlite:///chinook.db
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

# Работа с множествами

## Операция объединения

Вывести именя студентов со средним баллом, больше 3, или с именем, начинающимся с S.

In [23]:
%%sql 
SELECT name 
  FROM Students 
 WHERE name like 'S%'

 UNION 

SELECT name 
  FROM Students 
 WHERE gpa > 3

 * sqlite:///chinook.db
Done.


name
Lee
Petrov
Sidorov
Smith
Verikov


In [24]:
%%sql 
SELECT name 
  FROM Students 
 WHERE name like 'S%'

 UNION ALL

SELECT name 
  FROM Students 
 WHERE gpa > 3

 * sqlite:///chinook.db
Done.


name
Sidorov
Smith
Petrov
Verikov
Smith
Lee


Вывести имена студентов и имена сотрудников, и указать 1, если студент, 0, если сотрудник

In [25]:
%%sql
SELECT name, 
       1 as person_type
  FROM Students 
    
 UNION 

SELECT name,
       0 as person_type
  FROM Employee

 * sqlite:///chinook.db
Done.


name,person_type
Johnson,0
Jones,0
Lee,0
Lee,1
Petrov,1
Sidorov,1
Simon,0
Smith,0
Smith,1
Vasyugin,0


## Операция пересечения

Вывести такие имена, которые есть одновременно у сотрудников и студентов

In [28]:
%%sql
SELECT name
  FROM Students 
    
INTERSECT 

SELECT name
  FROM Employee

 * sqlite:///chinook.db
Done.


name
Lee
Smith


## Операция разности

Вывести имена, которые встречаются среди студентов, но которых нет у сотрудников

In [32]:
%%sql
SELECT name
  FROM Students 
    
EXCEPT 

SELECT name
  FROM Employee

 * sqlite:///chinook.db
Done.


name
Petrov
Sidorov
Verikov


# Работа с null

Вывести студентов, у которых не задан факультет

Неверно:

In [37]:
%%sql 
SELECT *
  FROM Students
 WHERE faculty_id = null

 * sqlite:///chinook.db
Done.


id,name,gpa,faculty_id


Верно:

In [39]:
%%sql 
SELECT *
  FROM Students
 WHERE faculty_id is null

 * sqlite:///chinook.db
Done.


id,name,gpa,faculty_id
6,Jiw,3.2,None


Вывести сотрудников, у которых нет вышестоящего менеджера

In [41]:
%%sql 
SELECT * 
  FROM employee
 WHERE manager_id is null

 * sqlite:///chinook.db
Done.


id,name,phone,title,manager_id
1,Vasyugin,+111,Director,None


Вывести сотрудников, у которых есть вышестоящий менеджер

In [43]:
%%sql 
SELECT * 
  FROM employee
 WHERE not (manager_id is null)

 * sqlite:///chinook.db
Done.


id,name,phone,title,manager_id
2,Jones,+112,Vice President,1
3,Smith,+113,Vice President,1
4,Velaskes,+114,Developer,2
5,Simon,+115,Analyst,2
6,Johnson,+116,Engineer,3
7,Lee,+117,Engineer,3


In [44]:
%%sql 
SELECT * 
  FROM employee
 WHERE manager_id is not null

 * sqlite:///chinook.db
Done.


id,name,phone,title,manager_id
2,Jones,+112,Vice President,1
3,Smith,+113,Vice President,1
4,Velaskes,+114,Developer,2
5,Simon,+115,Analyst,2
6,Johnson,+116,Engineer,3
7,Lee,+117,Engineer,3


## Вложенные подзапросы

### Поиск по множеств

Вывести студентов с факультета ИТ или Кибернетики

In [46]:
%%sql 
SELECT * 
  FROM Students 
 WHERE faculty_id in (SELECT id FROM Faculties WHERE name = 'IT' or name = 'KIB')

 * sqlite:///chinook.db
Done.


id,name,gpa,faculty_id
1,Sidorov,3,1
2,Petrov,3.5,2
3,Verikov,4,1
4,Smith,4.2,2
5,Lee,4.5,1


Вывести студентов, которые не учаться на факультете ИТ

In [47]:
%%sql 
SELECT * 
  FROM Students 
 WHERE faculty_id not in (SELECT id FROM Faculties WHERE name = 'IT')

 * sqlite:///chinook.db
Done.


id,name,gpa,faculty_id
2,Petrov,3.5,2
4,Smith,4.2,2


Заметьте, что есть еще один студент не с факультета ИТ

In [48]:
%%sql 
SELECT * 
  FROM Students 
 WHERE faculty_id not in (SELECT id FROM Faculties WHERE name = 'IT')
    OR faculty_id is null

 * sqlite:///chinook.db
Done.


id,name,gpa,faculty_id
2,Petrov,3.5,2
4,Smith,4.2,2
6,Jiw,3.2,None


Вывести студентов со средним баллом, большего чем средний балл у худшего студента с факультета Кибернетики

In [49]:
%%sql 
SELECT * 
  FROM Students 
 WHERE gpa > (
              SELECT MIN(gpa) 
                FROM Students 
               WHERE faculty_id = (select id FROM Faculties WHERE name = 'KIB'))

 * sqlite:///chinook.db
Done.


id,name,gpa,faculty_id
3,Verikov,4,1
4,Smith,4.2,2
5,Lee,4.5,1


Вывести студентов со средним баллом, большего чем средний балл у лучшего из худших струдентов по факультету

Как не стоит писать:

In [57]:
%%sql 
SELECT * 
  FROM Students 
 WHERE gpa > (
              SELECT MIN(gpa) 
                FROM Students 
               GROUP BY faculty_id)

 * sqlite:///chinook.db
Done.


id,name,gpa,faculty_id
2,Petrov,3.5,2
3,Verikov,4,1
4,Smith,4.2,2
5,Lee,4.5,1


In [58]:
%%sql 
SELECT * 
  FROM Students 
 WHERE gpa > (
              SELECT MIN(gpa) 
                FROM Students 
               GROUP BY faculty_id
               ORDER BY min(gpa) desc)

 * sqlite:///chinook.db
Done.


id,name,gpa,faculty_id
3,Verikov,4,1
4,Smith,4.2,2
5,Lee,4.5,1


Правильные варианты:

%%sql 
SELECT * 
  FROM Students 
 WHERE gpa > (SELECT max(gpa)
                FROM (
                     SELECT MIN(gpa) as gpa
                       FROM Students 
                      GROUP BY faculty_id))

In [61]:
%%sql 
SELECT * 
  FROM Students 
 WHERE gpa >any (
                     SELECT MIN(gpa) as gpa
                       FROM Students 
                      GROUP BY faculty_id)

 * sqlite:///chinook.db
(sqlite3.OperationalError) near "SELECT": syntax error
[SQL: SELECT * 
  FROM Students 
 WHERE gpa >any (
                     SELECT MIN(gpa) as gpa
                       FROM Students 
                      GROUP BY faculty_id)]
(Background on this error at: http://sqlalche.me/e/e3q8)


Вывести студентов со средним баллом, который больше чем минимальный средний балл на факультете

In [62]:
%%sql 
SELECT * 
  FROM Students s_o
 WHERE EXISTS(
    SELECT 1
      FROM Students s_i
     WHERE s_o.faculty_id = s_i.faculty_id
       AND s_o.gpa > s_i.gpa
    )

 * sqlite:///chinook.db
Done.


id,name,gpa,faculty_id
3,Verikov,4,1
4,Smith,4.2,2
5,Lee,4.5,1


In [65]:
%%sql
SELECT s.*
  FROM Students s
  INNER 
  JOIN 
      (
      SELECT faculty_id, min(gpa) as min_gpa
        FROM Students 
       GROUP
          BY faculty_id
      ) G
    ON s.faculty_id = g.faculty_id
 WHERE s.gpa > g.min_gpa

 * sqlite:///chinook.db
Done.


id,name,gpa,faculty_id
3,Verikov,4,1
4,Smith,4.2,2
5,Lee,4.5,1


## Оператор WITH

In [71]:
%%sql
WITH g AS 
(
SELECT faculty_id, min(gpa) as min_gpa
  FROM Students 
 GROUP 
    BY faculty_id
)
SELECT s.*
  FROM Students s
 INNER 
  JOIN g
    ON s.faculty_id = g.faculty_id
 WHERE s.gpa > g.min_gpa

 * sqlite:///chinook.db
Done.


id,name,gpa,faculty_id
3,Verikov,4,1
4,Smith,4.2,2
5,Lee,4.5,1


In [72]:
%%sql
WITH KIB_fac_id AS 
(
SELECT id
  FROM Faculties 
 WHERE name = 'KIB'
),
min_gpa as 
(
SELECT MIN(gpa) as min_gpa
  FROM Students 
 WHERE faculty_id = (SELECT id FROM KIB_fac_id)
)
SELECT * 
  FROM Students 
 WHERE gpa > (SELECT min_gpa from min_gpa)

 * sqlite:///chinook.db
Done.


id,name,gpa,faculty_id
3,Verikov,4,1
4,Smith,4.2,2
5,Lee,4.5,1


## Скалярные подзапросы

Вывести для каждого факультета количество студентов на факультете

In [74]:
%%sql
SELECT id, (SELECT count(*) FROM Students s WHERE s.faculty_id = f.id ) as cnt
  FROM Faculties f

 * sqlite:///chinook.db
Done.


id,cnt
1,3
2,2


# Соединения

In [76]:
%%sql
SELECT s.*, f.*
  FROM Students s 
NATURAL
  JOIN Faculties f

 * sqlite:///chinook.db
Done.


id,name,gpa,faculty_id,id_1,name_1


In [77]:
%%sql
SELECT s.*, f.* 
  FROM Students s 
 INNER 
  JOIN Faculties F 
    ON s.faculty_id = f.id

 * sqlite:///chinook.db
Done.


id,name,gpa,faculty_id,id_1,name_1
1,Sidorov,3,1,1,IT
2,Petrov,3.5,2,2,KIB
3,Verikov,4,1,1,IT
4,Smith,4.2,2,2,KIB
5,Lee,4.5,1,1,IT


In [78]:
%%sql
SELECT s.*, f.* 
  FROM Students s 
  LEFT
  JOIN Faculties F 
    ON s.faculty_id = f.id

 * sqlite:///chinook.db
Done.


id,name,gpa,faculty_id,id_1,name_1
1,Sidorov,3,1,1,IT
2,Petrov,3.5,2,2,KIB
3,Verikov,4,1,1,IT
4,Smith,4.2,2,2,KIB
5,Lee,4.5,1,1,IT
6,Jiw,3.2,None,None,None


In [79]:
%%sql
SELECT s.*, f.* 
  FROM Students s 
  LEFT
 OUTER
  JOIN Faculties F 
    ON s.faculty_id = f.id

 * sqlite:///chinook.db
Done.


id,name,gpa,faculty_id,id_1,name_1
1,Sidorov,3,1,1,IT
2,Petrov,3.5,2,2,KIB
3,Verikov,4,1,1,IT
4,Smith,4.2,2,2,KIB
5,Lee,4.5,1,1,IT
6,Jiw,3.2,None,None,None


In [80]:
%%sql
SELECT s.*, f.* 
  FROM Students s 
  LEFT
  JOIN Faculties F 
 USING (id)

 * sqlite:///chinook.db
Done.


id,name,gpa,faculty_id,id_1,name_1
1,Sidorov,3,1,1,IT
2,Petrov,3.5,2,2,KIB
3,Verikov,4,1,None,None
4,Smith,4.2,2,None,None
5,Lee,4.5,1,None,None
6,Jiw,3.2,None,None,None


# Представления

In [82]:
%%sql
CREATE VIEW StudentsAndFaculties AS
SELECT s.*, f.name as faculty_name 
  FROM Students s 
  LEFT
 OUTER
  JOIN Faculties F 
    ON s.faculty_id = f.id

 * sqlite:///chinook.db
Done.


[]

In [83]:
%%sql
SELECT * 
  FROM StudentsAndFaculties

 * sqlite:///chinook.db
Done.


id,name,gpa,faculty_id,faculty_name
1,Sidorov,3,1,IT
2,Petrov,3.5,2,KIB
3,Verikov,4,1,IT
4,Smith,4.2,2,KIB
5,Lee,4.5,1,IT
6,Jiw,3.2,None,None


## CASE WHEN

In [84]:
%%sql
SELECT CASE WHEN gpa > 4.0 
              THEN 'A'
            WHEN gpa > 3.0
              THEN 'B'
            ELSE 'C'
       END as mark
  FROM Students 

 * sqlite:///chinook.db
Done.


mark
C
B
B
A
A
B


In [85]:
%%sql
SELECT CASE WHEN gpa > 4.0 
              THEN 'A'
            WHEN gpa > 3.0
              THEN 'B'
            ELSE 'C'
       END as mark,
       count(*) as cnt
  FROM Students 
 GROUP 
    BY CASE WHEN gpa > 4.0 
              THEN 'A'
            WHEN gpa > 3.0
              THEN 'B'
            ELSE 'C'
       END

 * sqlite:///chinook.db
Done.


mark,cnt
A,2
B,3
C,1
